# Example using NGLUI for downloading and uploading annotations
This assumes an NGLUI install of version >4.0, which introduced breaking changes from previous versions of nglui

**The CAVEclient** is a python library that facilitates communication with a CAVE system. This package can can be installed with pip.

**NGLUI** is a python library the generates and parses neuroglancer links with data. For more detailed descriptions and examples, see the documenation here: https://www.caveconnecto.me/nglui/usage/statebuilder/

*Note:* when using colab, the package will have to be installed each session. If you run this in a local environmnet, the package needs to be installed just once.

In [ ]:
# @title Run this cell once to install and load packages
%%capture
!pip install -q caveclient
!pip install -q nglui

In [ ]:
from caveclient import CAVEclient
import pandas as pd
import numpy as np

# Makes pandas dataframes interactive (optional but useful)
from google.colab import data_table
data_table.enable_dataframe_formatter()

<b> CAVE account setup </b>

<p>In order to manage server traffic, every user needs to create a CAVE account and download a user token to access CAVE's services programmatically. The CAVE infrastructure can be read about in <a href='https://www.biorxiv.org/content/10.1101/2023.07.26.550598v1'>more detail on our preprint</a>. The MICrONS data is publicly available which means that no extra permissions need to be given to a new user account to access the data. Bulk downloads of some static data are also available without an account on <a href='https://microns-explorer.org/'> MICrONs Explorer</a>.

**A Google account (or Google-enabled account) is required to create a CAVE account.**

Go to: https://global.daf-apis.com/auth/api/v1/user/token to view a list of your existing tokens

If you have never made a token before, accept the terms of service and generate a token. Then copy the hash string and paste below


In [3]:
# @title Initialize CAVE
# @markdown Requires your unique user token. Input user token

client = CAVEclient()
user_token = '' # @param {type: "string"}

try:
  client.auth.save_token(token=user_token, overwrite=True)
  # Initialize a client for the production datastack.
  client = CAVEclient(datastack_name='minnie65_public')

  cg = client.chunkedgraph
  print('Client initiation successful!')
except:
  print('Oops! Token does not work')
  print('Go to: https://global.daf-apis.com/auth/api/v1/user/token to view a list of your existing tokens')
  print('If you have never made a token before: \n  1) go here: https://minnie.microns-daf.com/materialize/views/datastack/minnie65_public to accept terms of service \n  2) then go here https://global.daf-apis.com/auth/api/v1/create_token to create a new token.')


Client initiation successful!


The **NGLUI** `parser` module offers a number of tools to get information about neuroglancer states out of the JSON format that neuroglancer uses. The recommended approach here is to pass a dictionary representation of the JSON object the StateParser class and build various kinds of dataframes from it. We will use the `CAVEclient` class to get a state JSON based on its ID from the 'share' button, but you could also use the text you can download from the {} button in the viewer.

## Posting NGL states to the link shortener
In Neuroglancer, on the upper right, there is a 'Share' button.

When you click this link, you will be prompted for a google login. Use any google profile, but it is convenient to use the same as for this notebook.

This will upload the text of your neuroglancer state (the JSON format file, also accessed with the {} button) to a server, and returns a lookup number to access that state. The link with this state is automatically copied to your system's clipboard.

When you paste it, you will see something like this:

> https://ngl.microns-explorer.org/#!middleauth+https://global.daf-apis.com/nglstate/api/v1/6047968401555456

NLGUI can read the information directly out of that uploaded state with a tool called `Parser`:

In [4]:
from nglui import parser

client = CAVEclient('minnie65_public')
state_json = client.state.get_state_json(6047968401555456)
state = parser.StateParser(state_json)

You can now access different aspects of the state. For example, to get a list of all layers and their core info, you can use the `layer_dataframe` method.

In [5]:
state.layer_dataframe()

,layer,type,source,archived
0,img,image,precomputed://https://bossdb-open-data.s3.amaz...,False
1,seg,segmentation,precomputed://gs://iarpa_microns/minnie/minnie...,False
2,annotation,annotation,local://annotations,False


This will give you a table with a row for each layer and columns for layer name, type, source, and whether the layer is archived (i.e. visible) or not.

With `parser` you can get a list of all annotations with the annotation_dataframe method.

In [6]:
state.annotation_dataframe()

,layer,anno_type,point,pointB,linked_segmentation,tags,group_id,description
0,annotation,point,"[178032.171875, 153603.828125, 20031.5]",NaN,[],[0],None,None
1,annotation,point,"[177677.328125, 153394.09375, 20067.240234375]",NaN,[],[1],None,another qualitative label
2,annotation,point,"[176720.078125, 151505.296875, 20209.501953125]",NaN,[],[],None,None
3,annotation,point,"[178893.109375, 156596.671875, 20690.5]",NaN,[],[],None,qualitative label


This will give you a dataframe where each row is an annotation, and columns show `layer name, `point` locations, annotation type, annotation id, descriptive text, linked segmentations, tags, etc.

If you have multiple annotation layers, each layer is specified by the `layer` column and all points across all layers are concatenated together.

The point coordinates are returned at the resolution of the neuroglancer view (default: 4x4x40 nm). You can change this resolution with argument `point_resolution` which will rescale the points to the requested resolution.

The `description` column populates with any text you have attached to the point.

If you are using tags, the `expand_tags=True` argument will create a column for every tag and assign a boolean value to the row based on whether the tag is present in the annotation.

Another option that is sometimes useful is `split_points=True`, which will create a separate column for each x, y, or z coordinate in the annotation.

In [7]:
state.annotation_dataframe(expand_tags=True, split_points=True, point_resolution=[1,1,1])

,layer,anno_type,linked_segmentation,tags,group_id,description,point_x,point_y,point_z,pointB_x,pointB_y,pointB_z,tag-1,tag-2
0,annotation,point,[],[0],None,None,712128.6875,614415.3125,801260.000000,NaN,NaN,NaN,True,False
1,annotation,point,[],[1],None,another qualitative label,710709.3125,613576.3750,802689.609375,NaN,NaN,NaN,False,True
2,annotation,point,[],[],None,None,706880.3125,606021.1875,808380.078125,NaN,NaN,NaN,False,False
3,annotation,point,[],[],None,qualitative label,715572.4375,626386.6875,827620.000000,NaN,NaN,NaN,False,False


## Uploading local annotations to neuroglancer

**NGLUI** also has many functions for turning data into neuroglancer states. As a toy example, lets reupload the points from the previous example.

Here we include only the point and description field. Tags are handled differently. See the [NGLUI documentation](https://caveconnectome.github.io/nglui/usage/statebuilder/) for details.

In [19]:
data_df = state.annotation_dataframe()[['point','description']]
data_df

,point,description
0,"[178032.171875, 153603.828125, 20031.5]",None
1,"[177677.328125, 153394.09375, 20067.240234375]",another qualitative label
2,"[176720.078125, 151505.296875, 20209.501953125]",None
3,"[178893.109375, 156596.671875, 20690.5]",qualitative label


In [33]:
from nglui.statebuilder import ViewerState, ImageLayer, SegmentationLayer, AnnotationLayer
(
    ViewerState(dimensions=[4,4,40])
    # .add_layers_from_client(client, segmentation='seg') # Adds the defaults from the public datastack (up to date proofreading) which you may or may not want
    .add_layer(ImageLayer(source=state.layer_dataframe().source[0])) # Adds layers from example dataframe instead
    .add_layer(SegmentationLayer(source=state.layer_dataframe().source[1]))
    .add_layer(
        AnnotationLayer(
            name='new-annotations',
        )
        .add_points(
            data=data_df,
            point_column='point',
            description_column='description',
            data_resolution=[4,4,40],
        )
    )
).to_link()

0


/usr/local/lib/python3.12/dist-packages/nglui/statebuilder/source_info.py:68: UserWarning: CloudVolume is not available. Cannot get source information for precomputed://https://bossdb-open-data.s3.amazonaws.com/iarpa_microns/minnie/minnie65/em.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/nglui/statebuilder/source_info.py:68: UserWarning: CloudVolume is not available. Cannot get source information for precomputed://gs://iarpa_microns/minnie/minnie65/seg_m1300.
  warnings.warn(
